In [3]:
# Load and Split
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma

from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser,StrOutputParser

from langchain import hub
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.documents import Document

from ragas import EvaluationDataset, RunConfig, evaluate
from ragas.metrics import (
    LLMContextRecall, Faithfulness, LLMContextPrecisionWithReference, AnswerRelevancy
)
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper

from pydantic import BaseModel, Field

from textwrap import dedent
from operator import itemgetter
from pprint import pprint
import random

from dotenv import load_dotenv
load_dotenv()


True

In [4]:
COLLECTION_NAME = "medicine_docs"
DOC_PATH = 'data/medicine.txt'
# Text Loading
loader = TextLoader(DOC_PATH, encoding='utf-8')

# 문서 load and split
splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100
)

docs = loader.load_and_split(splitter)

len(docs)

4222

In [5]:
### 평가 데이터로 사용할 context 추출
total_samples = 3

# index shuffle 후 total_samples만큼 context 추출

idx_list = list(range(len(docs)))
random.shuffle(idx_list)

eval_context_list = []
while len(eval_context_list) < total_samples:
    idx = idx_list.pop()
    context = docs[idx].page_content
    if len(context) > 100: # 100글자 이상인 text만 사용
        eval_context_list.append(context)

len(eval_context_list)

3

In [6]:
# user_input: 질문
# ####### retrieved_contexts: 검색된 문서의 내용(page_content)들
# qa_context: 질문 답변 쌍을 만들 때 참고한 context
        # retrieved_contexts: 검색된 문서의 내용은 실제 RAG 실행시 넣는다.
        # response: 모델의 답변 - 실제 RAG 실행시 넣는다.
# reference: 정답
class EvalDatasetSchema(BaseModel):
    user_input: str = Field(..., title="질문(question)")
    qa_context: list[str] = Field(..., title="질문-답변 쌍을 만들 때 참조한 context.")
    reference: str = Field(..., title="질문의 정답(ground truth)")

parser = JsonOutputParser(pydantic_object=EvalDatasetSchema)

eval_model = ChatOpenAI(model="gpt-4o")
prompt_template = PromptTemplate.from_template(
    template=dedent("""
        당신은 RAG 평가를 위해 질문과 정답 쌍을 생성하는 인공지능 비서입니다.
        다음 [Context] 에 문서가 주어지면 해당 문서를 기반으로 {num_questions}개의 질문을 생성하세요. 

        질문과 정답을 생성한 후 아래의 출력 형식 GUIDE 에 맞게 생성합니다.
        질문은 반드시 [context] 문서에 있는 정보를 바탕으로 생성해야 합니다. [context]에 없는 내용을 가지고 질문-답변을 절대 만들면 안됩니다.
        질문은 간결하게 작성합니다.
        하나의 질문에는 한 가지씩만 내용만 작성합니다. 
        질문을 만들 때 "제공된 문맥에서", "문서에 설명된 대로", "주어진 문서에 따라" 또는 이와 유사한 말을 하지 마세요.
        질문을 만들 때, 구체적인 증상에 따른 약을 추천받는 질문만 하세요.
        구체적인 증상에 따른 약 추천받는 질문만 하세요. 제발. 다른 질문은 하지 마세요.
        질문을 할 때는 구체적인 약 이름을 기반으로 질문하세요. "이 약" 또는 이와 유사한 말을 사용하지 마세요.

        정답은 반드시 [context]에 있는 정보를 바탕으로 작성합니다. 없는 내용을 추가하지 않습니다.
        정답은 반드시 [context]에 있는 구체적인 약 이름을 명시해서 답변하세요. 
        정답에는 구체적인 약 이름으로 추천해서 답변하세요.
        정답에 "이 약"이라는 말을 쓰지 마세요. "이 약"이라는 말을 쓰지 마. 제발. 쓰지 말라면 좀 쓰지마.

        질문과 답변을 만들고 그 내용이 [context] 에 있는 항목인지 다시 한번 확인합니다.
        생성된 질문-답변 쌍은 반드시 dictionary 형태로 정의하고 list로 묶어서 반환해야 합니다.
        질문-답변 쌍은 반드시 {num_questions}개를 만들어 주십시오.

        출력 형식: {format_instructions}

        [Context]
        {context}
        """
    ),
    partial_variables={"format_instructions":parser.get_format_instructions()}
)
# print(prompt_template.template)

eval_dataset_generator = prompt_template | eval_model | parser

In [7]:
############################################################
# eval_context_list 모두로 만들기
# 
# 생성된 질문-답변을 눈으로 보고 검증한 및 수정해야 한다.
############################################################
eval_data_list = []
num_questions = 5
for context in eval_context_list:
    _eval_data_list = eval_dataset_generator.invoke({"context":context, "num_questions":num_questions})
    eval_data_list.extend(_eval_data_list)

In [8]:
import pandas as pd
eval_df = pd.DataFrame(eval_data_list)
eval_df.shape

(15, 3)

In [9]:
## 생성 된 질문/답 쌍 확인
eval_df.head()
eval_df.tail()

,user_input,qa_context,reference
10,육체피로 시 어떤 약을 복용할 수 있나요?,[키즈하이츄어블정],육체피로 시 키즈하이츄어블정을 복용할 수 있습니다.
11,임신 중 비타민 보충을 위해 어떤 약을 사용할 수 있나요?,[키즈하이츄어블정],임신 중 비타민 보충을 위해 키즈하이츄어블정을 사용할 수 있습니다.
12,야맹증 예방에 적합한 약은 무엇인가요?,[키즈하이츄어블정],야맹증 예방을 위해 키즈하이츄어블정을 사용할 수 있습니다.
13,뼈 발육 불량 개선을 위해 어떤 약을 추천하시나요?,[키즈하이츄어블정],뼈 발육 불량 개선에는 키즈하이츄어블정을 추천합니다.
14,"비타민 A, D, E 보급을 위한 약은 무엇인가요?",[키즈하이츄어블정],"비타민 A, D, E 보급을 위해 키즈하이츄어블정을 복용할 수 있습니다."


# Chain 구성

In [10]:
# Vector Store 연결
COLLECTION_NAME = "medicine_docs"
PERSIST_DIRECTORY = "vector_store/chroma/medicine_db"
embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")
vector_store = Chroma(
    embedding_function=embedding_model,
    collection_name=COLLECTION_NAME,
    persist_directory=PERSIST_DIRECTORY
)

In [11]:
vector_store._collection.count()

4222

In [12]:
# Chain 구성
# prompt template. langchain hub에 등록된 것을 가져와서 사용.
prompt_template = hub.pull("rlm/rag-prompt")
# prompt_template

# Retriever 생성
retriever = vector_store.as_retriever(
    search_type='mmr',
    search_kwargs={
        'k':3,
        'fetch_k':10,
        'lambda_mult':0.5
    }
)
model = ChatOpenAI(model="gpt-4o-mini")

def format_docs(src_docs:dict[str, list[Document]]) -> str:
    """list[Document]: Vector Store에서 검색한 context들에서 
    page_content만 추출해서 하나의 문자열로 합쳐서 반환"""
    docs = src_docs['context']
    return "\n\n".join([doc.page_content for doc in docs])

def str_from_documents(docs: list[Document]) -> list[str]:
    """list[Document]에서 page_content 값들만 추출한 list를 반환."""
    return [doc.page_content for doc in docs]

rag_chain = (
    RunnablePassthrough() # rag chain을 RunnableSequence로 만들기 위해 Runnable인 것으로 시작.
    | {
        "context": retriever, "question":RunnablePassthrough()
    } # retriver -> {"context":list[Document], "question":"user input"}
    | {
        # 앞에서 넘어온 dictionary에서 context(List[Document])를 추출 -> page_content값들을 list로 반환. list[str]
        "source_context" : itemgetter("context") | RunnableLambda(str_from_documents), 
        "llm_answer": {
            # {"context":list[Document]} -> str(page_content들만 모은 string)
            "context": RunnableLambda(format_docs), "question":itemgetter("question")
        } | prompt_template | model | StrOutputParser()  # LLM 응답 처리 chain. 
    }
)

In [13]:
eval_df['user_input']

0           위·십이지장궤양에 어떤 약을 사용할 수 있나요?
1        위염 증상을 완화하기 위해 추천되는 약은 무엇인가요?
2        위산과다로 고생 중인데 어떤 약을 복용하면 좋을까요?
3                  속쓰림에 효과적인 약은 무엇인가요?
4            구역과 구토에 도움이 되는 약을 알고 싶어요.
5     혈액 이상이 있는 환자가 복용할 수 있는 약은 무엇인가요?
6           간경화 환자가 복용할 수 있는 약은 무엇인가요?
7     신장 장애가 있는 환자가 복용할 수 있는 약은 무엇인가요?
8      심장 기능 부전 환자가 복용할 수 있는 약은 무엇인가요?
9           고혈압 환자가 복용할 수 있는 약은 무엇인가요?
10             육체피로 시 어떤 약을 복용할 수 있나요?
11    임신 중 비타민 보충을 위해 어떤 약을 사용할 수 있나요?
12               야맹증 예방에 적합한 약은 무엇인가요?
13        뼈 발육 불량 개선을 위해 어떤 약을 추천하시나요?
14        비타민 A, D, E 보급을 위한 약은 무엇인가요?
Name: user_input, dtype: object

In [14]:
# rag_chain에 평가 질문을 입력해서 context들과 모델답변을 응답 받아 eval_dataset(eval_df)에 추가.
context_list = []
response_list = []

for user_input in eval_df['user_input']:
    res = rag_chain.invoke(user_input)
    context_list.append(res['source_context'])
    response_list.append(res['llm_answer'])

In [15]:
print(len(context_list), len(response_list))
# pprint(context_list[:2])
# pprint(response_list[:2])

15 15


In [16]:
response_list

['위·십이지장궤양에는 이든알마게이트정, 일양바이오알마게이트정, 이탄칼정과 같은 약을 사용할 수 있습니다. 이들 약물은 제산작용 및 증상 개선에 효과적입니다. 복용 전 반드시 의사나 약사와 상담하는 것이 중요합니다.',
 '위염 증상을 완화하기 위해 추천되는 특정 약물에 대한 정보는 제공되지 않았습니다. 위염 증상 완화에 대한 약물은 일반적으로 의사와 상담 후 사용해야 합니다. 따라서 정확한 약물 정보는 전문가에게 문의하는 것이 좋습니다.',
 '위산과다로 고생 중이라면 위스타산이나 위푸린에스산을 고려해볼 수 있습니다. 두 약 모두 15세 이상 성인이 하루 3회, 식후 또는 식간에 복용하는 것이 좋습니다. 그러나 2주 이상 복용해도 증상이 개선되지 않으면 즉시 복용을 중단하고 의사 또는 약사와 상담해야 합니다.',
 '속쓰림에 효과적인 약으로는 속크린에스정이 있습니다. 이 약은 위산과다, 속쓰림, 소화불량 등의 증상에 사용됩니다. 복용 전 의사나 약사와 상담하는 것이 좋습니다.',
 '어떤 특정한 약에 대한 정보는 제공되지 않았습니다. 구역과 구토에 도움이 되는 약을 원하신다면 의사나 약사와 상담하는 것이 좋습니다. 그들은 귀하의 상황에 맞는 적절한 약을 추천해 줄 수 있습니다.',
 '혈액 이상이 있는 환자는 약 복용 전에 의사나 약사와 상의해야 합니다. 특정 약물은 이러한 환자에게 적합하지 않을 수 있으며, 출혈 경향이 있는 환자에게도 주의가 필요합니다. 따라서 정확한 약물은 전문가의 조언을 통해 결정해야 합니다.',
 '간경화 환자는 특정 약물을 복용하기 전에 반드시 의사 또는 약사와 상담해야 합니다. 일반적으로 간장애가 있는 환자는 피해야 할 약물이 많으므로, 적절한 치료와 약물 선택이 중요합니다. 구체적인 약물에 대한 정보는 의료 전문가에게 문의하는 것이 가장 좋습니다.',
 '신장 장애가 있는 환자는 특정 약을 복용하기 전에 반드시 의사 또는 약사와 상담해야 합니다. 이와 관련된 약물의 복용은 주의가 필요하며, 환자의 상태에 따라 다를 수 있습니다.

In [17]:
eval_df["retrieved_contexts"] = context_list # context 추가
eval_df["response"] = response_list   # 정답 추가

In [18]:
eval_df.head(3)

,user_input,qa_context,reference,retrieved_contexts,response
0,위·십이지장궤양에 어떤 약을 사용할 수 있나요?,[알바트정(알마게이트) (주)바이넥스 이 약은 위·십이지장궤양에 사용합니다.],위·십이지장궤양에는 알바트정(알마게이트)을 사용할 수 있습니다.,"[이든알마게이트정 (주)이든파마 이 약은 위·십이지장궤양, 위염, 위산과다, 속쓰림...","위·십이지장궤양에는 이든알마게이트정, 일양바이오알마게이트정, 이탄칼정과 같은 약을 ..."
1,위염 증상을 완화하기 위해 추천되는 약은 무엇인가요?,[알바트정(알마게이트) (주)바이넥스 이 약은 위염 증상의 개선에 사용합니다.],위염 증상을 완화하기 위해 알바트정(알마게이트)을 추천합니다.,"[센티렉스어드밴스정 헤일리온코리아주식회사 이 약은 육체피로, 임신ㆍ수유기, 병중ㆍ병...",위염 증상을 완화하기 위해 추천되는 특정 약물에 대한 정보는 제공되지 않았습니다. ...
2,위산과다로 고생 중인데 어떤 약을 복용하면 좋을까요?,[알바트정(알마게이트) (주)바이넥스 이 약은 위산과다 증상의 개선에 사용합니다.],위산과다에는 알바트정(알마게이트)을 복용하면 좋습니다.,"[위스타산 (주)아이월드제약 이 약은 과음, 속쓰림, 위트림, 과식, 위통, 위부불...",위산과다로 고생 중이라면 위스타산이나 위푸린에스산을 고려해볼 수 있습니다. 두 약 ...


# 평가

In [19]:
# Dataframe으로 부터 EvalDataset 생성
eval_dataset = EvaluationDataset.from_pandas(eval_df)
eval_dataset

EvaluationDataset(features=['user_input', 'retrieved_contexts', 'response', 'reference'], len=15)

In [20]:
# HuggingFace에 업로드 -> datasets.Dataset 으로 변환
eval_dataset.to_hf_dataset()#.push_to_hub()

Dataset({
    features: ['user_input', 'retrieved_contexts', 'response', 'reference'],
    num_rows: 15
})

In [21]:
# model_name = "gpt-4o"
model_name = "gpt-4o-mini"
model = ChatOpenAI(model=model_name)
eval_llm = LangchainLLMWrapper(model)

embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")
eval_embedding = LangchainEmbeddingsWrapper(embedding_model)


## GPT-4o-mini 모델을 사용하여 평가 
metrics = [
    LLMContextRecall(llm=eval_llm),
    LLMContextPrecisionWithReference(llm=eval_llm),
    Faithfulness(llm=eval_llm),
    AnswerRelevancy(llm=eval_llm, embeddings=eval_embedding)
]
result = evaluate(dataset=eval_dataset, metrics=metrics)

Evaluating:   0%|          | 0/60 [00:00<?, ?it/s]

In [22]:
result

{'context_recall': 0.3333, 'llm_context_precision_with_reference': 0.3611, 'faithfulness': 0.8471, 'answer_relevancy': 0.1401}